In [1]:
import streamlit as st 
from PyPDF2 import PdfReader
from langchain.embeddings import OpenAIEmbeddings, SentenceTransformerEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.memory import ConversationBufferWindowMemory
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
def get_pdf_text(pdf_path):
    text = ""
    pdf_reader = PdfReader(pdf_path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

#지정된 조건에 따라 주어진 텍스트를 더 작은 덩어리로 분할
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(
        separators="\\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

#주어진 텍스트 청크에 대한 임베딩을 생성하고 FAISS를 사용하여 벡터 저장소를 생성
def get_vectorstore(text_chunks):
    embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

#주어진 벡터 저장소로 대화 체인을 초기화
def get_conversation_chain(vectorstore):
    memory = ConversationBufferWindowMemory(memory_key='chat_history', return_message=True)  #ConversationBufferWindowMemory에 이전 대화 저장
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(temperature=0.7, model_name='gpt-3.5-turbo-16k-0613'),
        retriever=vectorstore.as_retriever(),
        get_chat_history=lambda h: h,
        memory=memory
    ) #ConversationalRetrievalChain을 통해 langchain 챗봇에 쿼리 전송
    return conversation_chain

In [4]:
# PDF 파일 경로 설정
pdf_path = "The_Adventures_of_Tom_Sawyer.pdf"
# PDF 텍스트 가져오기
raw_text = get_pdf_text(pdf_path)
# 텍스트에서 청크 검색
text_chunks = get_text_chunks(raw_text)
# PDF 텍스트 저장을 위해 FAISS 벡터 저장소 만들기
vectorstore = get_vectorstore(text_chunks)
# 대화 체인 만들기
m = get_conversation_chain(vectorstore)

C:\Users\wnghk\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\wnghk\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\wnghk\anaconda3\envs\llm\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpen

In [8]:
import speech_recognition as sr
import pyttsx3

# 음성 입력 (STT)
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("질문을 해주세요.")
        audio = recognizer.listen(source)
    try:
        question = recognizer.recognize_google(audio, language="ko-KR")
        print("사용자:", question)
        return question
    except sr.UnknownValueError:
        print("음성을 이해할 수 없습니다.")
        return ""
    except sr.RequestError:
        print("음성 서비스에 접근할 수 없습니다.")
        return ""

# 음성 출력 (TTS)
def speak_response(response):
    engine = pyttsx3.init()  # 기본 드라이버로 초기화
    engine.say(response)
    engine.runAndWait()

# 질문 처리 및 답변 생성
def process_question(question):
    # 질문에 대한 답변 가져오기
    response = m({"question": question})
    if 'answer' in response:
        answer = response['answer']
    else:
        print("올바른 응답 키를 찾을 수 없습니다.")
    return answer

if __name__ == "__main__":
    while True:
        question = recognize_speech()
        if question=="멈춰":
            break;
        if question:
            answer = process_question(question)
            print(answer)
            speak_response(answer)


질문을 해주세요.
사용자: 그럼 언제 죽어
주어진 텍스트에서는 주인공이 죽는 부분에 대한 언급이 없습니다. 따라서 주인공이 언제 죽는지에 대한 정보를 알 수 없습니다.
질문을 해주세요.
사용자: 여자 주인공 누구 있어
여자 주인공은 Becky입니다.
질문을 해주세요.
사용자: 백희 말고는 없어
주어진 정보에서는 백희 외에 다른 여자 주인공에 대한 언급이 없습니다.
질문을 해주세요.
사용자: 멈춰 멈춰 멈춰
저는 그 질문에 대한 답을 모릅니다.
질문을 해주세요.
사용자: 멈춰
